# libCEED for Python examples

This is a tutorial to illustrate the main feautures of the Python interface for [libCEED](https://github.com/CEED/libCEED/), the low-level API library for efficient high-order discretization methods developed by the co-design [Center for Efficient Exascale Discretizations](https://ceed.exascaleproject.org/) (CEED) of the [Exascale Computing Project](https://www.exascaleproject.org/) (ECP).

While libCEED's focus is on high-order finite/spectral element method implementations, the approach is mostly algebraic and thus applicable to other discretizations in factored form, as explained in the [user manual](https://libceed.org/).

## Setting up libCEED for Python

Install libCEED for Python by running

In [ ]:
! python -m pip install libceed

## CeedOperator

Here we show some basic examples to illustrate the `libceed.Operator` class. In libCEED, a `libceed.Operator` defines the finite/spectral element operator associated to a `libceed.QFunction` (see [the API documentation](https://libceed.org/en/latest/libCEEDapi.html#finite-element-operator-decomposition)).

* In the following example, we create and apply a CeedOperator for the mass matrix in 1D. By applying this operator to a vector of 1's, we compute the length of this 1D domain, similar to Ex1-Volume in the [tutorial-6-shell tutorial](./tutorial-6-shell.ipynb)

In [ ]:
import libceed
import numpy as np

ceed = libceed.Ceed()

num_elem = 15
p = 5
q = 8
num_x = num_elem + 1
num_u = num_elem*(p-1) + 1

# Vectors
x = ceed.Vector(num_x)
x_array = np.zeros(num_x)
for i in range(num_x):
  x_array[i] = i / (num_x - 1.0)
x.set_array(x_array, cmode=libceed.USE_POINTER)

q_data = ceed.Vector(num_elem*q)
u = ceed.Vector(num_u)
v = ceed.Vector(num_u)

# Restrictions
indices_x = np.zeros(num_x*2, dtype="int32")
for i in range(num_x):
  indices_x[2*i+0] = i
  indices_x[2*i+1] = i+1
restriction_x = ceed.ElemRestriction(num_elem, 2, 1, 1, num_x, indices_x, cmode=libceed.USE_POINTER)

indices_u = np.zeros(num_elem*p, dtype="int32")
for i in range(num_elem):
  for j in range(p):
    indices_u[p*i+j] = i*(p-1) + j
restriction_u = ceed.ElemRestriction(num_elem, p, 1, 1, num_u, indices_u, cmode=libceed.USE_POINTER)
strides = np.array([1, q, q], dtype="int32")
restriction_q_data = ceed.StridedElemRestriction(num_elem, q, 1, q*num_elem, strides)

# Bases
basis_x = ceed.BasisTensorH1Lagrange(1, 1, 2, q, libceed.GAUSS)
basis_u = ceed.BasisTensorH1Lagrange(1, 1, p, q, libceed.GAUSS)

# QFunctions
qf_setup = ceed.QFunctionByName("Mass1DBuild")
qf_mass = ceed.QFunctionByName("MassApply")

# Setup operator
op_setup = ceed.Operator(qf_setup)
op_setup.set_field("dx", restriction_x, basis_x, libceed.VECTOR_ACTIVE)
op_setup.set_field("weights", libceed.ELEMRESTRICTION_NONE, basis_x,
                   libceed.VECTOR_NONE)
op_setup.set_field("qdata", restriction_q_data, libceed.BASIS_NONE,
                   libceed.VECTOR_ACTIVE)
op_setup.check()
print('Setup operator: ', op_setup)

# Mass operator
op_mass = ceed.Operator(qf_mass)
op_mass.set_field("u", restriction_u, basis_u, libceed.VECTOR_ACTIVE)
op_mass.set_field("qdata", restriction_q_data, libceed.BASIS_NONE, q_data)
op_mass.set_field("v", restriction_u, basis_u, libceed.VECTOR_ACTIVE)
op_mass.check()
print('Mass operator: ', op_mass)

# Setup
op_setup.apply(x, q_data)

# Apply mass matrix
u.set_value(1)
op_mass.apply(u, v)

# Check
with v.array_read() as v_array:
  print('The length of the domain is l = %4.2f'%np.sum(v_array))

* In the next example, we create and apply a CeedOperator for the Poisson operator in 1D. By applying this operator to a vector with a linear function, we compute the 'surface area' of this 1D domain, similar to Ex2-Surface in the [tutorial-6-shell tutorial](./tutorial-6-shell.ipynb)

In [ ]:
import libceed
import numpy as np

ceed = libceed.Ceed()

num_elem = 15
p = 5
q = 8
num_x = num_elem + 1
num_u = num_elem*(p-1) + 1

# Vectors
x = ceed.Vector(num_x)
x_array = np.zeros(num_x)
for i in range(num_x):
  x_array[i] = i / (num_x - 1.0)
x.set_array(x_array, cmode=libceed.USE_POINTER)

q_data = ceed.Vector(num_elem*q)
u = ceed.Vector(num_u)
v = ceed.Vector(num_u)

# Restrictions
indices_x = np.zeros(num_x*2, dtype="int32")
for i in range(num_x):
  indices_x[2*i+0] = i
  indices_x[2*i+1] = i+1
restriction_x = ceed.ElemRestriction(num_elem, 2, 1, 1, num_x, indices_x, cmode=libceed.USE_POINTER)

indices_u = np.zeros(num_elem*p, dtype="int32")
for i in range(num_elem):
  for j in range(p):
    indices_u[p*i+j] = i*(p-1) + j
restriction_u = ceed.ElemRestriction(num_elem, p, 1, 1, num_u, indices_u, cmode=libceed.USE_POINTER)
strides = np.array([1, q, q], dtype="int32")
restriction_q_data = ceed.StridedElemRestriction(num_elem, q, 1, q*num_elem, strides)

# Bases
basis_x = ceed.BasisTensorH1Lagrange(1, 1, 2, q, libceed.GAUSS)
basis_u = ceed.BasisTensorH1Lagrange(1, 1, p, q, libceed.GAUSS)

# QFunctions
qf_setup = ceed.QFunctionByName("Poisson1DBuild")
qf_mass = ceed.QFunctionByName("Poisson1DApply")

# Setup operator
op_setup = ceed.Operator(qf_setup)
op_setup.set_field("dx", restriction_x, basis_x, libceed.VECTOR_ACTIVE)
op_setup.set_field("weights", libceed.ELEMRESTRICTION_NONE, basis_x,
                   libceed.VECTOR_NONE)
op_setup.set_field("qdata", restriction_q_data, libceed.BASIS_NONE,
                   libceed.VECTOR_ACTIVE)
op_setup.check()
print('Setup operator: ', op_setup)

# Poisson operator
op_poisson = ceed.Operator(qf_mass)
op_poisson.set_field("du", restriction_u, basis_u, libceed.VECTOR_ACTIVE)
op_poisson.set_field("qdata", restriction_q_data, libceed.BASIS_NONE, q_data)
op_poisson.set_field("dv", restriction_u, basis_u, libceed.VECTOR_ACTIVE)
op_poisson.check()
print('Poisson operator: ', op_poisson)

# Setup
op_setup.apply(x, q_data)

# Apply Poisson operator
with u.array_write() as u_array:
  [points, _] = ceed.lobatto_quadrature(p)
  for elem in range(num_elem):
      for point in range(p):
          u_array[elem * (p - 1) + point] = (1.0 + 2.0 * elem + points[point])/(2.0 * num_elem)
op_poisson.apply(u, v)

# Check
with v.array_read() as v_array:
  print('The surface area of the domain is dl = %4.2f'%np.sum(abs(v_array)))